In [1]:
from __future__ import annotations

import asyncio
import nest_asyncio

# make asyncio.run() works in notebook
nest_asyncio.apply()

def load_plaintext(filename):
    with open(filename, 'r', encoding="UTF-8") as f:
        return f.read()

# ~/.metagpt/config2.yaml
from metagpt.config2 import config
config.llm

2024-03-19 15:33:49.901 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /Users/deryzhou/Downloads/StreamChatPlayground/notebooks


LLMConfig(extra_fields=None, api_key='2089197edd83eddf547f600665da02e9.ohYNy9csNMqduxID', api_type=<LLMType.ZHIPUAI: 'zhipuai'>, base_url='https://api.openai.com/v1', api_version=None, model='glm-4', pricing_plan=None, access_key=None, secret_key=None, endpoint=None, app_id=None, api_secret=None, domain=None, max_token=4096, temperature=0.0, top_p=1.0, top_k=5, repetition_penalty=1.0, stop=None, presence_penalty=0.0, frequency_penalty=0.0, best_of=None, n=None, stream=True, logprobs=None, top_logprobs=None, timeout=60, proxy=None, calc_usage=True)

In [2]:
import sys
from metagpt.logs import logger
from metagpt.provider.zhipuai_api import ZhiPuAILLM

logger.remove()
logger.add(sys.stderr, level="DEBUG")
llm = ZhiPuAILLM(config.llm)
llm.use_system_prompt = False # Disable default system message

asyncio.run(llm.aask("你好"))

## 任务拆解

In [44]:
import os
from langchain.prompts import PromptTemplate
# from langchain_experimental.plan_and_execute import PlanAndExecute

path = "prompts/planning"

task_types = """
**web scraping**: For scraping data from web pages.
"""

output_format = load_plaintext(os.path.join(path, "output.md"))

prompt = PromptTemplate(
    input_variables=[],
    template=load_plaintext(os.path.join(path, "template.yaml")),
)
prompt = prompt.partial(
    task_types=task_types,
    max_tasks=20,
    output=output_format,
)
# print(prompt.format(context="这里是指令"))

In [45]:
goal = """抓取 https://pitchhub.36kr.com/financing-flash 中'快讯'的内容，并整理成markdown存档"""

user_guidance = """大概的流程：
- 使用工具抓取网页中的可见文本
- 提取网页文本中'快讯'相关的内容。注意网页中可能包含导航，只需要抽取'快讯'的具体内容
- 对抽取结果进行归类，并保存成markdown表格: 快讯.md"""

planning_prompt = prompt.format(goal=goal, user_guidance=user_guidance)
# print(planning_prompt + "\n\n")

rsp = asyncio.run(llm.aask(msg=planning_prompt))

2024-03-19 11:05:27.558 | DEBUG    | metagpt.provider.base_llm:aask:126 - [{'role': 'user', 'content': 'Respond to the human as helpfully and accurately as possible.\n\n# User goal\n抓取 https://pitchhub.36kr.com/financing-flash 中\'快讯\'的内容，并整理成markdown存档\n大概的流程：\n- 使用工具抓取网页中的可见文本\n- 提取网页文本中\'快讯\'相关的内容。注意网页中可能包含导航，只需要抽取\'快讯\'的具体内容\n- 对抽取结果进行归类，并保存成markdown表格: 快讯.md\n\n# Available Task Types:\n**web scraping**: For scraping data from web pages.\n\n# Task:\nBased on the user goal, write a plan or modify an existing plan of what you should do to achieve the goal. A plan consists of one to 20 tasks.\nIf you are modifying an existing plan, carefully follow the instruction, don\'t make unnecessary changes. Give the whole plan unless instructed to modify only one task of the plan.\nIf you encounter errors on the current task, revise and output the current single task only.\nOutput a list of jsons following the format:\n```json\n[\n    {{\n        "task_id": str = "unique identifier for a task in

[
    {
        "task_id": "1",
        "dependent_task_ids": [],
        "instruction": "Visit the webpage https://pitchhub.36kr.com/financing-flash in a web browser.",
        "task_type": "web scraping"
    },
    {
        "task_id": "2",
        "dependent_task_ids": ["1"],
        "instruction": "Identify the sections of the webpage containing the visible text related to '快讯'.",
        "task_type": "web scraping"
    },
    {
        "task_id": "3",
        "dependent_task_ids": ["2"],
        "instruction": "Extract the visible text from these identified sections, excluding navigation and other non-essential content.",
        "task_type": "web scraping"
    },
    {
        "task_id": "4",
        "dependent_task_ids": ["3"],
        "instruction": "Parse the extracted text to separate individual '快讯' entries.",
        "task_type": "web scraping"
    },
    {
        "task_id": "5",
        "dependent_task_ids": ["4"],
        "instruction": "Organize the parsed entries into 

In [46]:
import json
from metagpt.utils.common import OutputParser

try:
    tasks = json.loads(rsp)
except:
    code_block = OutputParser.parse_code(rsp, "json")
    tasks = json.loads(code_block)

tasks

[{'task_id': '1',
  'dependent_task_ids': [],
  'instruction': 'Visit the webpage https://pitchhub.36kr.com/financing-flash in a web browser.',
  'task_type': 'web scraping'},
 {'task_id': '2',
  'dependent_task_ids': ['1'],
  'instruction': "Identify the sections of the webpage containing the visible text related to '快讯'.",
  'task_type': 'web scraping'},
 {'task_id': '3',
  'dependent_task_ids': ['2'],
  'instruction': 'Extract the visible text from these identified sections, excluding navigation and other non-essential content.',
  'task_type': 'web scraping'},
 {'task_id': '4',
  'dependent_task_ids': ['3'],
  'instruction': "Parse the extracted text to separate individual '快讯' entries.",
  'task_type': 'web scraping'},
 {'task_id': '5',
  'dependent_task_ids': ['4'],
  'instruction': 'Organize the parsed entries into a structured format suitable for a markdown table.',
  'task_type': 'web scraping'},
 {'task_id': '6',
  'dependent_task_ids': ['5'],
  'instruction': "Save the struc

## 任务执行

In [47]:
from metagpt.actions.di.execute_nb_code import ExecuteNbCode

pre_execute = """import asyncio
import nest_asyncio
nest_asyncio.apply()
"""

execute_code = ExecuteNbCode()
await execute_code.run(pre_execute)

  1 import asyncio                                                                                                 
  2 import nest_asyncio                                                                                            
  3 nest_asyncio.apply()                                                                                           
  4                                                                                                                

('', True)

In [52]:
path = "prompts/task_plan"

task_types = """**web scraping**: For scraping data from web pages."""

output_format = load_plaintext(os.path.join(path, "output.md"))

prompt = PromptTemplate(
    input_variables=[],
    template=load_plaintext(os.path.join(path, "template.yaml")),
)
prompt = prompt.partial(
    plan_status="",
    current_task=tasks[0]['instruction'],
    task_guidance="",
    tools="""{'scrape_web_playwright': {'type': 'async_function', 'description': 'Asynchronously Scrape and save the HTML structure and inner text content of a web page using Playwright. ', 'signature': '(url)', 'parameters': "Args: url (str): The main URL to fetch inner text from. Returns: dict: The inner text content and html structure of the web page, keys are 'inner_text', 'html'.", 'tool_path': 'metagpt/tools/libs/web_scraping.py'}}""",
    output=output_format,
)
# print(prompt.format(goal="这里是指令"))

In [ ]:
task_prompt = prompt.format(goal=goal)
print(task_prompt + "\n\n")

rsp = asyncio.run(llm.aask(msg=task_prompt))
code = OutputParser.parse_code(rsp, "python")
print(code)

In [55]:
result, success = await execute_code.run(code)
result, success

   1 # Import the tool                                                                                             
   2 from metagpt.tools.libs.web_scraping import scrape_web_playwright                                             
   3                                                                                                               
   4 # Define the URL to be scraped                                                                                
   5 url = 'https://pitchhub.36kr.com/financing-flash'                                                             
   6                                                                                                               
   7 # Use the tool to scrape the webpage                                                                          
   8 # Note: Since this is an async function, we use 'await' in a notebook environment.                            
   9 # It is assumed that the notebook environment is set up to handle await properly.                             
  10 result = await scrape_web_playwright(url)                                                                     
  11                                                                                                               
  12 # The result contains 'inner_text' and 'html' keys.                                                           
  13 # You can access them as follows:                                                                             
  14 inner_text = result['inner_text']                                                                             
  15 html = result['html']                                                                                         
  16                                                                                                               
  17 # If you need to display the results:                                                                         
  18 print("Inner Text:\n", inner_text)                                                                            
  19 print("\nHTML:\n", html)                                                                                      
  20                                                                                                               

(',Inner Text:\n 首页\n融资快报\n融资事件\n项目库\n机构库\n项目集\n定向对接\n融通创新\n公司/项目名/投资机构/赛道\n\xa0\n返回36氪\n登录\n融资快报\n快讯\n“智谱AI”已完成新一轮融资\n36氪获悉，2024 年初，“智谱AI”已完成新一轮融资，北京市人工智能产业投资基金参与投资。北京市人工智能产业投资基金致力于加快推动北京市人工智能产业高质量发展，支持底层关键核心技术突破，促进适合首都定位的重大创新成果在京落地产业化，营造人工智能产业创新生态。智谱AI表示，本次融资将持续用于基座大模型的进一步研发，与各家产业龙头企业合作，一同在千行百业打造蓬勃繁荣的大模型生态，共同赋能落实市政府产业战略，助力北京国际科技创新中心建设，加速迈向通用人工智能的时代。\n智谱AI\n战略融资\n北京市\n2019年成立\nAI知识智能技术开发商\n8分钟前\n文章\n「深知未来」再获数千万融资，推动AI夜视产品走向全球市场\n实现AI夜视增强技术规模化商用\n深知未来\n股权融资\n广东省\n2017年成立\n人工智能全彩夜视产品研发商\n53分钟前\n肖千平\n快讯\n“中科加禾”获数千万元融资\n36氪获悉，“中科加禾”近日完成数千万元天使轮融资，本轮由元禾原点领投，新尚资本、中科院创投、晨山资本、BV百度风投跟投，指数资本继续担任独家财务顾问。本轮融资资金计划用于产品研发和公司运营。\n中科加禾\n天使轮\n江苏省\n2023年成立\nAI 生态软件基础设施供应商\n1小时前\n文章\n打破美国垄断，「昱瓴新能源」A+轮融资获航盛锂能领投 | 36氪首发\n专注高性能锂电负极材料。\n1小时前\n张卓倩\n快讯\n中州创赢获数千万人民币A轮融资\n安徽省中州创赢大数据科技有限公司（简称“中州创赢”）近日宣布完成数千万人民币A轮融资，投资方未披露。中州创赢是一家AIGC服务提供商，公司成立于2020年1月，位于安徽合肥。\n2小时前\n文章\n解决芯片编译技术“卡脖子”问题，「中科加禾」获数千万元融资｜智涌首发\n国内最早做编译器研发的团队之一。\n中科加禾\n天使轮\n江苏省\n2023年成立\nAI 生态软件基础设施供应商\n2小时前\n武静静Jane\n快讯\n「天一晶能」完成新一轮股权融资\n工商信息变更显示，3月12日，成

In [14]:
# if success = False
# error = result
error = "The stdout will print to user directly, if you want return anthing, just `print()` it."

path = "prompts/task_plan"

task_types = """**web scraping**: For scraping data from web pages."""

output_format = load_plaintext(os.path.join(path, "output.md"))

plan_status = PromptTemplate(
    input_variables=[],
    template=load_plaintext(os.path.join(path, "plan_status.yaml")),
)

prompt = PromptTemplate(
    input_variables=[],
    template=load_plaintext(os.path.join(path, "template.yaml")),
)
prompt = prompt.partial(
    plan_status=plan_status.format(error=error, code=code),
    current_task=tasks[0]['instruction'],
    task_guidance="analyze your previous code and error in follow step by step, provide me with improved code.",
    tools="""{'scrape_web_playwright': {'type': 'async_function', 'description': 'Asynchronously Scrape and save the HTML structure and inner text content of a web page using Playwright. ', 'signature': '(url)', 'parameters': "Args: url (str): The main URL to fetch inner text from. Returns: dict: The inner text content and html structure of the web page, keys are 'inner_text', 'html'.", 'tool_path': 'metagpt/tools/libs/web_scraping.py'}}""",
    output=output_format,
)
# print(prompt.format(goal="这里是指令"))

In [56]:
print(result)

,Inner Text:
 首页
融资快报
融资事件
项目库
机构库
项目集
定向对接
融通创新
公司/项目名/投资机构/赛道
 
返回36氪
登录
融资快报
快讯
“智谱AI”已完成新一轮融资
36氪获悉，2024 年初，“智谱AI”已完成新一轮融资，北京市人工智能产业投资基金参与投资。北京市人工智能产业投资基金致力于加快推动北京市人工智能产业高质量发展，支持底层关键核心技术突破，促进适合首都定位的重大创新成果在京落地产业化，营造人工智能产业创新生态。智谱AI表示，本次融资将持续用于基座大模型的进一步研发，与各家产业龙头企业合作，一同在千行百业打造蓬勃繁荣的大模型生态，共同赋能落实市政府产业战略，助力北京国际科技创新中心建设，加速迈向通用人工智能的时代。
智谱AI
战略融资
北京市
2019年成立
AI知识智能技术开发商
8分钟前
文章
「深知未来」再获数千万融资，推动AI夜视产品走向全球市场
实现AI夜视增强技术规模化商用
深知未来
股权融资
广东省
2017年成立
人工智能全彩夜视产品研发商
53分钟前
肖千平
快讯
“中科加禾”获数千万元融资
36氪获悉，“中科加禾”近日完成数千万元天使轮融资，本轮由元禾原点领投，新尚资本、中科院创投、晨山资本、BV百度风投跟投，指数资本继续担任独家财务顾问。本轮融资资金计划用于产品研发和公司运营。
中科加禾
天使轮
江苏省
2023年成立
AI 生态软件基础设施供应商
1小时前
文章
打破美国垄断，「昱瓴新能源」A+轮融资获航盛锂能领投 | 36氪首发
专注高性能锂电负极材料。
1小时前
张卓倩
快讯
中州创赢获数千万人民币A轮融资
安徽省中州创赢大数据科技有限公司（简称“中州创赢”）近日宣布完成数千万人民币A轮融资，投资方未披露。中州创赢是一家AIGC服务提供商，公司成立于2020年1月，位于安徽合肥。
2小时前
文章
解决芯片编译技术“卡脖子”问题，「中科加禾」获数千万元融资｜智涌首发
国内最早做编译器研发的团队之一。
中科加禾
天使轮
江苏省
2023年成立
AI 生态软件基础设施供应商
2小时前
武静静Jane
快讯
「天一晶能」完成新一轮股权融资
工商信息变更显示，3月12日，成都天一晶能半导体有限公司（以下简称为“天一晶能”）完成新一轮股权融资，新进股东为君鼎私募基金、大衍私募基金、芯微智汇。「天一晶能」是一个

In [ ]:
path = "prompts/extract_data"

completed_tasks = "- **{}**: {}\n".format(tasks[0]['task_type'], tasks[0]['instruction'])

current_task = """当前任务是抽取页面文本内容，并按指定格式输出。请留意下面几点：

- 页面主体部分是一个列表，每个`快讯`后面是需要提取的关键信息
- 请不要漏掉或补充不在提供页面内容中的信息，完全按照提供的内容抽取
- 网页中可能包含导航，因此头尾部分无关内容可以舍弃
- 最后，请对抽取的内容生成字段的表头"""

output_format = load_plaintext(os.path.join(path, "output_markdown_list.md"))

prompt = PromptTemplate(
    input_variables=[],
    template=load_plaintext(os.path.join(path, "template.yaml")),
)
prompt = prompt.partial(
    completed_tasks=completed_tasks,
    content=result,
    current_task=current_task,
    output=output_format,
)
# print(prompt.format(goal=goal))

In [60]:
task_prompt = prompt.format(goal=goal)
print(task_prompt + "\n\n")

rsp = asyncio.run(llm.aask(msg=task_prompt))
print(rsp)

2024-03-19 11:25:20.456 | DEBUG    | metagpt.provider.base_llm:aask:126 - [{'role': 'user', 'content': "As an AI Engineer, you need to help user to achieve their goal step by step.\n\n# 目标\n抓取 https://pitchhub.36kr.com/financing-flash 中'快讯'的内容，并整理成markdown存档\n\n## 已完成任务\n- **web scraping**: Visit the webpage https://pitchhub.36kr.com/financing-flash in a web browser.\n\n\n### 最后任务输出\n,Inner Text:\n 首页\n融资快报\n融资事件\n项目库\n机构库\n项目集\n定向对接\n融通创新\n公司/项目名/投资机构/赛道\n\xa0\n返回36氪\n登录\n融资快报\n快讯\n“智谱AI”已完成新一轮融资\n36氪获悉，2024 年初，“智谱AI”已完成新一轮融资，北京市人工智能产业投资基金参与投资。北京市人工智能产业投资基金致力于加快推动北京市人工智能产业高质量发展，支持底层关键核心技术突破，促进适合首都定位的重大创新成果在京落地产业化，营造人工智能产业创新生态。智谱AI表示，本次融资将持续用于基座大模型的进一步研发，与各家产业龙头企业合作，一同在千行百业打造蓬勃繁荣的大模型生态，共同赋能落实市政府产业战略，助力北京国际科技创新中心建设，加速迈向通用人工智能的时代。\n智谱AI\n战略融资\n北京市\n2019年成立\nAI知识智能技术开发商\n8分钟前\n文章\n「深知未来」再获数千万融资，推动AI夜视产品走向全球市场\n实现AI夜视增强技术规模化商用\n深知未来\n股权融资\n广东省\n2017年成立\n人工智能全彩夜视产品研发商\n53分钟前\n肖千平\n快讯\n“中科加禾”获数千万元融资\n36氪获悉，“中科加禾”近日完成数千万元天使轮融资，本轮由元禾原点领投，新尚资本、中科院创投、晨山资本、BV百度风投跟投，指数资本继续担任独家财务顾问。本轮融资资金计划用于产品研发和公

As an AI Engineer, you need to help user to achieve their goal step by step.

# 目标
抓取 https://pitchhub.36kr.com/financing-flash 中'快讯'的内容，并整理成markdown存档

## 已完成任务
- **web scraping**: Visit the webpage https://pitchhub.36kr.com/financing-flash in a web browser.


### 最后任务输出
,Inner Text:
 首页
融资快报
融资事件
项目库
机构库
项目集
定向对接
融通创新
公司/项目名/投资机构/赛道
 
返回36氪
登录
融资快报
快讯
“智谱AI”已完成新一轮融资
36氪获悉，2024 年初，“智谱AI”已完成新一轮融资，北京市人工智能产业投资基金参与投资。北京市人工智能产业投资基金致力于加快推动北京市人工智能产业高质量发展，支持底层关键核心技术突破，促进适合首都定位的重大创新成果在京落地产业化，营造人工智能产业创新生态。智谱AI表示，本次融资将持续用于基座大模型的进一步研发，与各家产业龙头企业合作，一同在千行百业打造蓬勃繁荣的大模型生态，共同赋能落实市政府产业战略，助力北京国际科技创新中心建设，加速迈向通用人工智能的时代。
智谱AI
战略融资
北京市
2019年成立
AI知识智能技术开发商
8分钟前
文章
「深知未来」再获数千万融资，推动AI夜视产品走向全球市场
实现AI夜视增强技术规模化商用
深知未来
股权融资
广东省
2017年成立
人工智能全彩夜视产品研发商
53分钟前
肖千平
快讯
“中科加禾”获数千万元融资
36氪获悉，“中科加禾”近日完成数千万元天使轮融资，本轮由元禾原点领投，新尚资本、中科院创投、晨山资本、BV百度风投跟投，指数资本继续担任独家财务顾问。本轮融资资金计划用于产品研发和公司运营。
中科加禾
天使轮
江苏省
2023年成立
AI 生态软件基础设施供应商
1小时前
文章
打破美国垄断，「昱瓴新能源」A+轮融资获航盛锂能领投 | 36氪首发
专注高性能锂电负极材料。
1小时前
张卓倩
快讯
中州创赢获数千万人民币A轮融资
安徽省中州创赢大数据科技有限公司（简称“中州创赢”）近

In [61]:
block = OutputParser.parse_code(rsp, "markdown")
print(block)

| 公司名称   | 融资轮次 | 所在地区 | 成立时间 | 公司简介                                   | 发布时间   | 文章链接       |
|-----------|---------|---------|----------|--------------------------------------------|------------|----------------|
| 智谱AI     | 战略融资 | 北京市  | 2019年   | AI知识智能技术开发商                       | 8分钟前    | 无             |
| 深知未来   | 股权融资 | 广东省  | 2017年   | 人工智能全彩夜视产品研发商                 | 53分钟前   | 无             |
| 中科加禾   | 天使轮   | 江苏省  | 2023年   | AI生态软件基础设施供应商                   | 1小时前    | 无             |
| 中州创赢   | A轮     | 安徽省  | 2020年   | AIGC服务提供商                             | 2小时前    | 无             |
| 天一晶能   | 股权融资 | 成都市  | 未提供   | 半导体分立器件销售商                       | 21小时前   | 无             |
| 纽瑞特医疗 | C轮     | 成都市  | 未提供   | 放射性药物研发、创新与产业化高新技术企业     | 23小时前   | 无             |
| 赛美特     | C轮     | 上海市  | 2017年   | 国产半导体智能制造软件供应商               | 昨天       | 无             |
| 光本位     | 天使+轮 | 未提供  | 未提供   | 光子计算芯片OPU研发商                      | 昨天       | 无             |
| 诺视科技   | Pre-A+轮 |